# Optimize code


## 1. Use matrix instead of for loop.


In [14]:
using LinearAlgebra

In [19]:
# original function with nMarker times for loop
function f1()
    nMarker = 50_000;
    nSample = 10_000;
    yCorr   = randn(nSample);
    xArray  = [rand(nSample) for i in 1:nMarker];
    xpx     = randn(nMarker).^2;
    α       = randn(nMarker);
    nLoci   = 100;
    δ       = ones(nMarker);

    for j=1:nMarker
        x = xArray[j];
        rhs = (dot(x,yCorr) + xpx[j]*α[j])*1.0

        lhs = xpx[j]*1.0 + 1.0
        invLhs = 1.0/lhs
        gHat   = rhs*invLhs
        logDelta1  = -0.5*(log(lhs) + 1.0 - gHat*rhs) - 2.3
        probDelta1 = 1.0/(1.0 + exp(-0.1 - logDelta1))
        oldAlpha   = α[j] 

        if(rand()<probDelta1)
            δ[j] = 1
            α[j] = gHat + randn()*sqrt(invLhs)
            BLAS.axpy!(oldAlpha-α[j],x,yCorr)
            nLoci = nLoci + 1
        else
            if (oldAlpha!=0)
                BLAS.axpy!(oldAlpha,x,yCorr)
            end
            δ[j] = 0
            α[j] = 0
        end
    end
end;

In [20]:
#running time for f1()
@time f1()

  6.501496 seconds (2.13 M allocations: 3.806 GiB, 67.26% gc time)


### 1.1 running time for f1()
6.000376 seconds (100.02 k allocations: 3.731 GiB, 70.60% gc time)  
6.069780 seconds (100.02 k allocations: 3.731 GiB, 69.76% gc time)  
5.855608 seconds (100.02 k allocations: 3.731 GiB, 69.73% gc time)  
Note: too much garbage collection time.

In [33]:
varinfo() #didn't use memory, because variables in f1() stored in stack.

| name        |       size | summary         |
|:----------- | ----------:|:--------------- |
| Base        |            | Module          |
| Core        |            | Module          |
| Main        |            | Module          |
| f           |    0 bytes | typeof(f)       |
| f1          |    0 bytes | typeof(f1)      |
| f2          |    0 bytes | typeof(f2)      |
| rng         | 18.879 KiB | MersenneTwister |
| startupfile |   54 bytes | String          |


### 1.2 Re-write via matrix

#### column major convention
Julia uses column major convention, which means reading column by column is much faster than row by row.  
test code:  
```{r}
function t1()
    x = Array{Float64,2}(undef,10_000,10_000)
    for i = 1:size(x,1)
        x1 = x[i,:] 
    end
end;

@time t1()  #1.37s

function t2()
    x = Array{Float64,2}(undef,10_000,10_000)
    for i = 1:size(x,2)
        x1 = x[:,i] 
    end
end;
@time t2()  # 0.55s
```

In [15]:
# function without loop
function f2()
    nMarker = 50_000;
    nSample = 10_000;
    yCorr   = randn(nSample, 1); # (10_00, 1)
    xArray  = rand(nSample,nMarker); #0.02s  (10_00, 50_00)
    xpx     = randn(nMarker,1).^2; #(50_00, 1)
    α       = randn(nMarker,1); #(50_00, 1)
    nLoci   = 100;
    δ       = ones(nMarker,1); #(50_00, 1)
    rhs = (yCorr' * xArray)' + xpx.*α;   #(50_00, 1)
    lhs = xpx*1.0 .+ 1.0; #(50_00, 1)
    invLhs = 1.0 ./ lhs;  #(50_00, 1)
    gHat   = rhs.*invLhs; #(50_00, 1)
    logDelta1  = -0.5 * (log.(lhs) .+ 1.0 .- gHat.*rhs) .- 2.3; #(50_00, 1)
    probDelta1 = 1.0 ./ (1.0 .+ exp.(-0.1 .- logDelta1)); #(50_00, 1)
    id = rand(nMarker,1).<probDelta1; #(50_00, 1)
    δ = δ .* id; #(50_00, 1)
    nLoci = nLoci+sum(id); #(50_00, 1)
    α_temp = gHat .+ randn(nMarker,1) .* sqrt.(invLhs); #(50_00, 1)
    α_new = α_temp .* id; #(50_00, 1)
    diffs = (α - α_new)'; #(1, 50_00)

    xArray = xArray .* diffs;#(10_00, 50_00)      <---- double memory

    sum_x_new = sum(xArray,dims = 2); #(10_00, 1)
    yCorr = yCorr .+ sum_x_new; #(10_00, 1)
    return 0
end;

In [17]:
@time f2()

  4.451809 seconds (68 allocations: 7.459 GiB, 18.49% gc time)


0

### 1.2 running time for f2()

4.298493 seconds (68 allocations: 7.459 GiB, 17.86% gc time)  
4.476440 seconds (68 allocations: 7.459 GiB, 18.66% gc time)  
4.451809 seconds (68 allocations: 7.459 GiB, 18.49% gc time)

We can see using matrix manipulation **decrease time, but double the memory**.

In [27]:
varinfo()

| name        |       size | summary         |
|:----------- | ----------:|:--------------- |
| Base        |            | Module          |
| Core        |            | Module          |
| Main        |            | Module          |
| f           |    0 bytes | typeof(f)       |
| f1          |    0 bytes | typeof(f1)      |
| f2          |    0 bytes | typeof(f2)      |
| rng         | 18.879 KiB | MersenneTwister |
| startupfile |   54 bytes | String          |


## 2. reduce gc time

We see that f1() has almost 70% gc time, which is too much. We need to decrease it.

### 2.1 running time for creating big matrix

In [7]:
function t(nMarker = 50_000, nSample = 10_000)
    xArray  = [rand(nSample) for i in 1:nMarker];
    return 0
end;

In [12]:
@time t()

  5.731432 seconds (100.01 k allocations: 3.729 GiB, 74.43% gc time)


0

In [38]:
function t2(nMarker = 50_000, nSample = 10_000)
    xArray  = rand(nSample,nMarker);
    return 0
end;

In [41]:
@time t2()

  1.768944 seconds (6 allocations: 3.725 GiB, 22.88% gc time)


0

time:  
5.731432 seconds (100.01 k allocations: 3.729 GiB, 74.43% gc time)  
5.728274 seconds (100.01 k allocations: 3.729 GiB, 74.90% gc time)  
5.832911 seconds (169.40 k allocations: 3.733 GiB, 74.21% gc time)  
Create big matrix is very time consuming.

In [13]:
varinfo()

| name        |     size | summary   |
|:----------- | --------:|:--------- |
| Base        |          | Module    |
| Core        |          | Module    |
| Main        |          | Module    |
| startupfile | 54 bytes | String    |
| t           |  0 bytes | typeof(t) |


### 2.2 Avoid create big matrix

In [18]:
function f3()
    nMarker = 50_000
    nSample = 10_000
    #xArray  = [rand(nSample) for i in 1:nMarker];   <--- avoid create big matrix
    yCorr   = randn(nSample);
    xpx     = randn(nMarker).^2;
    α       = randn(nMarker);
    nLoci   = 100
    δ       = ones(nMarker)
#for i= 1:50_000
    for j=1:nMarker
        x = rand(nSample)                        # <--- create small piece
        rhs = (dot(x,yCorr) + xpx[j]*α[j])*1.0

        lhs = xpx[j]*1.0 + 1.0
        invLhs = 1.0/lhs
        gHat   = rhs*invLhs
        logDelta1  = -0.5*(log(lhs) + 1.0 - gHat*rhs) - 2.3
        probDelta1 = 1.0/(1.0 + exp(-0.1 - logDelta1))
        oldAlpha   = α[j]

        if(rand()<probDelta1)
            δ[j] = 1
            α[j] = gHat + randn()*sqrt(invLhs)
            BLAS.axpy!(oldAlpha-α[j],x,yCorr)
            nLoci = nLoci + 1
        else
            if (oldAlpha!=0)
                BLAS.axpy!(oldAlpha,x,yCorr)
            end
            δ[j] = 0
            α[j] = 0
        end
    end
end;

### 2.3 running time for f3()

In [22]:
@time f3()

  1.146152 seconds (100.02 k allocations: 3.731 GiB, 18.52% gc time)


1.158405 seconds (100.02 k allocations: 3.731 GiB, 18.31% gc time)  
1.159397 seconds (100.02 k allocations: 3.731 GiB, 18.25% gc time)  
1.146152 seconds (100.02 k allocations: 3.731 GiB, 18.52% gc time)

We can see the time decreases dramatically. So we continue our optimization based on f3(). 

## 3. Parallel
based on f3().

### 3.1 optimize dot product

#### 3.1.1 running time for dot product

In [15]:
using LinearAlgebra
function t()
    nSample = 10_000;
    yCorr   = randn(nSample);
    x = rand(nSample); 
    
    @time rhs = dot(x,yCorr);

end;

In [22]:
t();

  0.000004 seconds


In [29]:
# another test
using LinearAlgebra
v1 = randn(10_000_000);
v2 = randn(10_000_000);

In [32]:
@time dot(v1,v2);

  0.006143 seconds (5 allocations: 176 bytes)


We can see the running time is very small, do we really need to speed up?  
So I also test parallel vector multiplication.

#### 3.1.2 shared array

In [1]:
using Distributed
using SharedArrays

In [2]:
addprocs(4)

4-element Array{Int64,1}:
 2
 3
 4
 5

In [3]:
nprocs()

5

In [4]:
function t2()
    nSample = 10_000;
    yCorr   = randn(nSample);
    x = rand(nSample);

    x_s= convert(SharedArray,x);
    y_s = convert(SharedArray,yCorr);

    @time @distributed (+) for i in 1:nSample
          x_s[i]*y_s[i]
    end
end;

In [16]:
t2()

  0.000607 seconds (609 allocations: 55.094 KiB)


-61.56809346014772

From above, if we use split every element for parallel, the time increases. We don't need parallel here.

### 3.2 optimize `BLAS.axpy!(oldAlpha-α[j],x,yCorr)`

#### 3.2.1 running time

In [33]:
function f5()
    nMarker = 50_000
    nSample = 10_000
    yCorr   = randn(nSample);
    xpx     = randn(nMarker).^2;
    α       = randn(nMarker);
    nLoci   = 100
    δ       = ones(nMarker)

    for j=1:10
        x = rand(nSample) 
        rhs = (dot(x,yCorr) + xpx[j]*α[j])*1.0

        lhs = xpx[j]*1.0 + 1.0
        invLhs = 1.0/lhs
        gHat   = rhs*invLhs
        logDelta1  = -0.5*(log(lhs) + 1.0 - gHat*rhs) - 2.3
        probDelta1 = 1.0/(1.0 + exp(-0.1 - logDelta1))
        oldAlpha   = α[j]

        if(rand()<probDelta1)
            δ[j] = 1
            α[j] = gHat + randn()*sqrt(invLhs)
            @time BLAS.axpy!(oldAlpha-α[j],x,yCorr)
            nLoci = nLoci + 1
        else
            if (oldAlpha!=0)
                BLAS.axpy!(oldAlpha,x,yCorr)
            end
            δ[j] = 0
            α[j] = 0
        end
    end
end;

In [35]:
f5()

  0.000004 seconds
  0.000003 seconds
  0.000002 seconds
  0.000008 seconds
  0.000005 seconds
  0.000004 seconds
  0.000004 seconds
  0.000004 seconds
  0.000004 seconds
  0.000004 seconds


There is no need to speed up.

### Multiple-threads

`export JULIA_NUM_THREADS=10`  

`Threads.nthreads()`

In [ ]:
using LinearAlgebra

function f3()
    nMarker = 50_000
    nSample = 10_000
    yCorr   = randn(nSample);
    xpx     = randn(nMarker).^2;
    α       = randn(nMarker);
    nLoci   = 100
    δ       = ones(nMarker)

    Threads.@threads for j=1:nMarker
        x = rand(nSample) 
        rhs = (dot(x,yCorr) + xpx[j]*α[j])*1.0

        lhs = xpx[j]*1.0 + 1.0
        invLhs = 1.0/lhs
        gHat   = rhs*invLhs
        logDelta1  = -0.5*(log(lhs) + 1.0 - gHat*rhs) - 2.3
        probDelta1 = 1.0/(1.0 + exp(-0.1 - logDelta1))
        oldAlpha   = α[j]

        if(rand()<probDelta1)
            δ[j] = 1
            α[j] = gHat + randn()*sqrt(invLhs)
            BLAS.axpy!(oldAlpha-α[j],x,yCorr)
            nLoci = nLoci + 1
        else
            if (oldAlpha!=0)
                BLAS.axpy!(oldAlpha,x,yCorr)
            end
            δ[j] = 0
            α[j] = 0
        end
    end
end;

`@time f3()`

Running time 10 threads:  
0.706303 seconds (97.59 k allocations: 3.643 GiB)  
0.991138 seconds (97.83 k allocations: 3.655 GiB, 10.72% gc time)  
0.929544 seconds (97.13 k allocations: 3.652 GiB, 8.05% gc time)  
Unstable time, but time did decreased.

Running time 20 threads:
3.457470 seconds (97.19 k allocations: 3.651 GiB, 40.40% gc time)  
2.483308 seconds (97.55 k allocations: 3.667 GiB, 46.88% gc time)  
1.734155 seconds (97.66 k allocations: 3.674 GiB, 14.09% gc time)  
Unstable time

If I use 40 threads, the time is unstable too, often around 4s.